In [ ]:
import requests
import time
from urllib.parse import urlencode

class DigitalOceanClient():
    def __init__(self, key):
        self.url = 'https://api.digitalocean.com/v2/%s'
        self.headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer ' + key,
            'Cache-control': 'no-cache'
        }

    def request(self, path):
        """
        Sends an HTTP GET request.
        Returns a yielded iteration. 
        example: next(request('droplets'))
        """
        url = self.url % path
        last = False
        while not last:
            r = requests.get(url, headers=self.headers).json()
            yield r
            if 'links' in r and 'pages' in r['links'] and 'next' in r['links']['pages']:
                url = r['links']['pages']['next']
            else:
                last = True
    
    def post(self, path, payload):
        """
        Sends an HTTP POST request.
        example: post('account/keys', {"name":"My SSH Public Key","public_key":"ssh-rsa AA..."})
        """
        url = self.url % path
        return requests.post(url, json=payload, headers=self.headers).json()
    
    def add_new_ssh_key(self, key_name, key_file):
        with open(key_file) as f:
            public_key = f.read()
            result = do.post('account/keys', {
                "name":key_name,
                "public_key":public_key
            })
            if 'ssh_key' in result:
                return result['ssh_key']['id']
        return False
    
    def get_key_id_by_name(self, key_name):
        for key in next(self.request('account/keys'))['ssh_keys']:
            if key['name'] == key_name:
                return key['id']
        return False
    
    def create_a_new_droplet(self, ssh_key_id, name="Torrent", region="sgp1", size="1gb"):
        print("Creating new droplet: {0} at {1} region..".format(name, region))
        result = do.post('droplets', {
            "name": name, 
            "region": region, 
            "size": size, 
            "image": "ubuntu-14-04-x64", 
            "ssh_keys": [ssh_key_id]
        })
        droplet_id = result['droplet']['id']
        print("Droplet id: {0}".format(result['droplet']['id']))
        print("Action id: {0}".format(result['links']['actions'][0]['id']))
        if 'links' in result and 'actions' in result['links']:
            while True:
                time.sleep(5)
                action = next(do.request('actions/%s' % result['links']['actions'][0]['id']))['action']
                print("Create: {0}".format(action['status']))
                if action['status'] != 'in-progress':
                    break
        ip_address = next(do.request('droplets/%s' % droplet_id))['droplet']['networks']['v4'][0]['ip_address']
        print("Droplet ip_address: {0}".format(ip_address))
        return droplet_id
    

In [ ]:
import os
import json
import random
from fabric import Connection

def deploy_torrent(host, key_file):
    print("Deploying torrent on : {0} ..".format(host))
    c = Connection(
        host=host,
        user="root",
        connect_kwargs={
            "key_filename": key_file,
        },
    )
    c.sudo('uname -a')
    c.sudo('apt-get update -q', hide='stderr')
    c.sudo('apt-get upgrade -q -y', hide='stderr')
    c.sudo('apt-get install -q -y transmission-cli transmission-common transmission-daemon', hide='stderr')
    c.sudo('service transmission-daemon stop')
    c.sudo('cp /etc/transmission-daemon/settings.json /etc/transmission-daemon/settings.orign')
    fab_result = c.sudo('cat /etc/transmission-daemon/settings.json', hide=True)
    default_settings = json.loads(fab_result.stdout)
    overlay_settings = {
      "rpc-authentication-required": True,
      "rpc-bind-address": "0.0.0.0",
      "rpc-enabled": True,
      "rpc-port": 9091,
      "rpc-url": "/transmission/",
      "rpc-username": "transmission",
      "rpc-whitelist": "*",
      "rpc-whitelist-enabled": False
    }
    with open('./settings.json') as data_file:
        overlay_settings = json.load(data_file)
        overlay_settings['rpc-password'] = "".join(random.sample("abcdefghijklmnopqrstuvwxyz01234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ",8))
    for (i, item) in overlay_settings.items():
        default_settings[i] = item
    print ("|=================================|")
    print ("|=== rpc-password: {0} ======|".format(default_settings['rpc-password']))
    print ("|=================================|")
    with open('./final_settings.json', "w") as f:
        json.dump(default_settings, f, indent=4)
    c.put('./final_settings.json', remote='/tmp/settings.json')
    c.sudo('cat /tmp/settings.json > /etc/transmission-daemon/settings.json')
    c.sudo('service transmission-daemon restart', hide='stderr')
    print ("\n\n|= web access:  http://transmission:{0}@{1}:{2}/  =|\n\n".
       format(default_settings['rpc-password'], c.host, default_settings['rpc-port']))
    print ("probably your scp command - after downloads - looks like this:\n\n")
    print ("\tscp -i {0} -r root@{1}:/var/lib/transmission-daemon/downloads/ ./".\
    format(c.connect_kwargs['key_filename'], c.host))

In [ ]:
do = DigitalOceanClient(input('DigitalOcean API Key: '))

In [ ]:
# generate rsa key pair
# linux example:
# $ HOSTNAME=`hostname` ssh-keygen -t rsa -C "$HOSTNAME" -f "./do_rsa" -P ""
key_name = "Key for temp node"
key_id = do.add_new_ssh_key(key_name, './do_rsa.pub')

In [ ]:
# key_id = do.get_key_id_by_name(key_name)
droplet_id = do.create_a_new_droplet(key_id, name="Torrent", region="sgp1", size="1gb")

In [ ]:
host = next(do.request('droplets/%s' % droplet_id))['droplet']['networks']['v4'][0]['ip_address']
deploy_torrent(host, os.getcwd() + '/do_rsa')